# Day 03: Model Development for Capstone

## Week 24 - Capstone Project

### Learning Objectives
- Build a complete ML model pipeline for trading signal generation
- Implement multiple model architectures (Linear, Tree, Neural Network)
- Create ensemble methods for robust predictions
- Apply proper cross-validation for time series
- Track experiments and model versioning

### Why Model Development Matters
In production quant systems, model development is **iterative and disciplined**:
- Start simple (baseline models)
- Add complexity only when justified
- Validate rigorously with time-series aware methods
- Document everything for reproducibility

In [ ]:
# Core Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Data
import yfinance as yf

# ML Libraries
from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Typing
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from abc import ABC, abstractmethod

np.random.seed(42)
torch.manual_seed(42)

print("✅ All libraries loaded successfully!")

## 1. Data Preparation

In [ ]:
# Configuration
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'JPM', 'GS']
START_DATE = '2019-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')

print(f"📥 Downloading data for {len(TICKERS)} tickers...")
data = yf.download(TICKERS, start=START_DATE, end=END_DATE, progress=False, auto_adjust=True)
prices = data['Close'].dropna()

print(f"✅ Data loaded: {prices.shape[0]} days, {prices.shape[1]} tickers")
print(f"📅 Date range: {prices.index[0].strftime('%Y-%m-%d')} to {prices.index[-1].strftime('%Y-%m-%d')}")

In [ ]:
def create_features(prices: pd.DataFrame, ticker: str) -> pd.DataFrame:
    """Create features for ML model."""
    df = pd.DataFrame(index=prices.index)
    price = prices[ticker]
    
    # Returns
    df['returns'] = price.pct_change()
    df['log_returns'] = np.log(price / price.shift(1))
    
    # Moving Averages
    for window in [5, 10, 20, 50]:
        df[f'sma_{window}'] = price.rolling(window).mean()
        df[f'returns_sma_{window}'] = df['returns'].rolling(window).mean()
    
    # Volatility
    for window in [10, 20, 60]:
        df[f'volatility_{window}'] = df['returns'].rolling(window).std()
    
    # Momentum
    for window in [5, 10, 20]:
        df[f'momentum_{window}'] = price / price.shift(window) - 1
    
    # RSI
    delta = price.diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    df['rsi_14'] = 100 - (100 / (1 + rs))
    
    # MACD
    ema12 = price.ewm(span=12).mean()
    ema26 = price.ewm(span=26).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    
    # Target: Next day return direction
    df['target'] = (df['returns'].shift(-1) > 0).astype(int)
    
    # Price relative to moving averages
    df['price_sma_20_ratio'] = price / df['sma_20']
    df['price_sma_50_ratio'] = price / df['sma_50']
    
    return df.dropna()

# Create features for AAPL as example
features_df = create_features(prices, 'AAPL')
print(f"\n📊 Features created: {features_df.shape[1]} columns, {features_df.shape[0]} rows")
print(f"\nFeature columns: {list(features_df.columns)}")

## 2. Model Architecture Framework

In [ ]:
@dataclass
class ModelMetrics:
    """Container for model evaluation metrics."""
    accuracy: float
    precision: float
    recall: float
    f1: float
    auc_roc: float
    
    def to_dict(self) -> dict:
        return {
            'accuracy': self.accuracy,
            'precision': self.precision,
            'recall': self.recall,
            'f1': self.f1,
            'auc_roc': self.auc_roc
        }


class BaseModel(ABC):
    """Abstract base class for all models."""
    
    @abstractmethod
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        pass
    
    @abstractmethod
    def predict(self, X: np.ndarray) -> np.ndarray:
        pass
    
    @abstractmethod
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        pass
    
    def evaluate(self, X: np.ndarray, y: np.ndarray) -> ModelMetrics:
        """Evaluate model performance."""
        y_pred = self.predict(X)
        y_proba = self.predict_proba(X)[:, 1]
        
        return ModelMetrics(
            accuracy=accuracy_score(y, y_pred),
            precision=precision_score(y, y_pred, zero_division=0),
            recall=recall_score(y, y_pred, zero_division=0),
            f1=f1_score(y, y_pred, zero_division=0),
            auc_roc=roc_auc_score(y, y_proba)
        )


class LogisticModel(BaseModel):
    """Logistic Regression baseline model."""
    
    def __init__(self, C: float = 1.0):
        self.model = LogisticRegression(C=C, max_iter=1000, random_state=42)
        self.scaler = StandardScaler()
        self.name = "Logistic Regression"
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        X_scaled = self.scaler.fit_transform(X)
        self.model.fit(X_scaled, y)
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        X_scaled = self.scaler.transform(X)
        return self.model.predict(X_scaled)
    
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        X_scaled = self.scaler.transform(X)
        return self.model.predict_proba(X_scaled)


class RandomForestModel(BaseModel):
    """Random Forest model."""
    
    def __init__(self, n_estimators: int = 100, max_depth: int = 10):
        self.model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42,
            n_jobs=-1
        )
        self.scaler = StandardScaler()
        self.name = "Random Forest"
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        X_scaled = self.scaler.fit_transform(X)
        self.model.fit(X_scaled, y)
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        X_scaled = self.scaler.transform(X)
        return self.model.predict(X_scaled)
    
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        X_scaled = self.scaler.transform(X)
        return self.model.predict_proba(X_scaled)


class GradientBoostModel(BaseModel):
    """Gradient Boosting model."""
    
    def __init__(self, n_estimators: int = 100, learning_rate: float = 0.1):
        self.model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=5,
            random_state=42
        )
        self.scaler = StandardScaler()
        self.name = "Gradient Boosting"
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        X_scaled = self.scaler.fit_transform(X)
        self.model.fit(X_scaled, y)
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        X_scaled = self.scaler.transform(X)
        return self.model.predict(X_scaled)
    
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        X_scaled = self.scaler.transform(X)
        return self.model.predict_proba(X_scaled)

print("✅ Model classes defined!")

## 3. Neural Network Model

In [ ]:
class TradingMLP(nn.Module):
    """Multi-layer perceptron for trading signals."""
    
    def __init__(self, input_dim: int, hidden_dims: List[int] = [64, 32], dropout: float = 0.3):
        super().__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, 2))  # Binary classification
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.network(x)


class NeuralNetModel(BaseModel):
    """Neural Network wrapper for trading."""
    
    def __init__(self, input_dim: int, hidden_dims: List[int] = [64, 32], 
                 epochs: int = 100, lr: float = 0.001, batch_size: int = 32):
        self.model = TradingMLP(input_dim, hidden_dims)
        self.scaler = StandardScaler()
        self.epochs = epochs
        self.lr = lr
        self.batch_size = batch_size
        self.name = "Neural Network"
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        X_scaled = self.scaler.fit_transform(X)
        
        X_tensor = torch.FloatTensor(X_scaled).to(self.device)
        y_tensor = torch.LongTensor(y).to(self.device)
        
        dataset = TensorDataset(X_tensor, y_tensor)
        loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)
        
        self.model.to(self.device)
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        criterion = nn.CrossEntropyLoss()
        
        self.model.train()
        for epoch in range(self.epochs):
            for X_batch, y_batch in loader:
                optimizer.zero_grad()
                outputs = self.model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        self.model.eval()
        X_scaled = self.scaler.transform(X)
        X_tensor = torch.FloatTensor(X_scaled).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(X_tensor)
            _, predicted = torch.max(outputs, 1)
        
        return predicted.cpu().numpy()
    
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        self.model.eval()
        X_scaled = self.scaler.transform(X)
        X_tensor = torch.FloatTensor(X_scaled).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(X_tensor)
            probs = torch.softmax(outputs, dim=1)
        
        return probs.cpu().numpy()

print("✅ Neural Network model defined!")

## 4. Time Series Cross-Validation

In [ ]:
def time_series_cv(model: BaseModel, X: np.ndarray, y: np.ndarray, 
                   n_splits: int = 5) -> Dict[str, List[float]]:
    """
    Perform time series cross-validation.
    
    Uses expanding window to ensure no future data leakage.
    """
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    results = {
        'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'auc_roc': []
    }
    
    for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        # Create fresh model instance
        if isinstance(model, LogisticModel):
            fold_model = LogisticModel()
        elif isinstance(model, RandomForestModel):
            fold_model = RandomForestModel()
        elif isinstance(model, GradientBoostModel):
            fold_model = GradientBoostModel()
        else:
            fold_model = model
        
        fold_model.fit(X_train, y_train)
        metrics = fold_model.evaluate(X_test, y_test)
        
        for key, value in metrics.to_dict().items():
            results[key].append(value)
    
    return results

# Prepare data for cross-validation
feature_cols = [c for c in features_df.columns if c not in ['target', 'returns', 'log_returns']]
X = features_df[feature_cols].values
y = features_df['target'].values

print(f"📊 Feature matrix shape: {X.shape}")
print(f"📊 Target distribution: {np.bincount(y.astype(int))}")

## 5. Model Comparison

In [ ]:
# Initialize models
models = [
    LogisticModel(),
    RandomForestModel(),
    GradientBoostModel()
]

# Run cross-validation for each model
results_summary = {}

print("\n" + "="*70)
print("MODEL COMPARISON - Time Series Cross-Validation")
print("="*70)

for model in models:
    print(f"\n⏳ Evaluating {model.name}...")
    cv_results = time_series_cv(model, X, y, n_splits=5)
    
    results_summary[model.name] = {
        metric: f"{np.mean(values):.4f} ± {np.std(values):.4f}"
        for metric, values in cv_results.items()
    }
    
    print(f"   Accuracy: {np.mean(cv_results['accuracy']):.4f} ± {np.std(cv_results['accuracy']):.4f}")
    print(f"   AUC-ROC:  {np.mean(cv_results['auc_roc']):.4f} ± {np.std(cv_results['auc_roc']):.4f}")
    print(f"   F1 Score: {np.mean(cv_results['f1']):.4f} ± {np.std(cv_results['f1']):.4f}")

# Display summary table
print("\n" + "="*70)
print("SUMMARY TABLE")
print("="*70)
summary_df = pd.DataFrame(results_summary).T
print(summary_df.to_string())

## 6. Train Final Model & Generate Signals

In [ ]:
# Train final model on all data
final_model = GradientBoostModel(n_estimators=100, learning_rate=0.1)
final_model.fit(X, y)

# Generate predictions for all tickers
print("\n" + "="*70)
print("🎯 TRADING SIGNALS FOR ALL TICKERS")
print("="*70)
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}\n")

all_signals = {}

for ticker in TICKERS:
    ticker_features = create_features(prices, ticker)
    feature_cols = [c for c in ticker_features.columns if c not in ['target', 'returns', 'log_returns']]
    
    # Get last row for prediction
    X_last = ticker_features[feature_cols].iloc[-1:].values
    
    # Predict probability
    prob = final_model.predict_proba(X_last)[0, 1]
    
    # Determine signal
    if prob > 0.60:
        signal = "🟢 STRONG BUY"
        action = "Consider buying shares or CALL options"
    elif prob > 0.55:
        signal = "🟡 WEAK BUY"
        action = "Consider small position"
    elif prob < 0.40:
        signal = "🔴 STRONG SELL"
        action = "Consider selling or PUT options"
    elif prob < 0.45:
        signal = "🟠 WEAK SELL"
        action = "Consider reducing position"
    else:
        signal = "⚪ HOLD"
        action = "No clear edge - maintain position"
    
    all_signals[ticker] = {
        'probability': prob,
        'signal': signal,
        'action': action,
        'price': prices[ticker].iloc[-1]
    }
    
    print(f"{'='*30} {ticker} {'='*30}")
    print(f"   Current Price: ${prices[ticker].iloc[-1]:.2f}")
    print(f"   Up Probability: {prob:.2%}")
    print(f"   Signal: {signal}")
    print(f"   Action: {action}")
    print()

## 7. Key Takeaways

### What We Built Today:
1. **Feature Engineering Pipeline**: Technical indicators, momentum, volatility
2. **Multiple Models**: Logistic Regression, Random Forest, Gradient Boosting
3. **Time Series CV**: Proper validation without data leakage
4. **Signal Generation**: Probability-based trading signals for all tickers

### Tomorrow's Preview: Backtesting Framework
- Full backtesting with transaction costs
- Walk-forward optimization
- Risk metrics (Sharpe, Sortino, Max Drawdown)